In [1]:
from tradealgorithm.tradealgorithm import TradeAlgorithm
from tradealgorithm.tradealgorithmfactory import TradeAlgorithmFactory
from pricer.pricer_factory import PricerFactory
from pricer.pricer import Pricer
from ranker.ranker_factory import RankerFactory
from ranker.ranker import Ranker
from classifier.classifier_factory import ClassifierFactory
from classifier.classifier import Classifier
from datetime import datetime
from tqdm import tqdm

In [2]:
trade_algo = TradeAlgorithmFactory.build(TradeAlgorithm.RRR_BETA)

In [3]:
ranker_class = Ranker.NONE
classifier_class = Classifier.NONE
current = False

In [4]:
start = datetime(2001,1,1)
end = datetime(2023,1,1)
current_date = datetime.now()

In [5]:
pricer_classes = [] 
# pricer_classes.append(Pricer.WEEKLY_STOCK_WINDOW)
pricer_classes.append(Pricer.DAILY_STOCK_ROLLING)

In [6]:
for pricer_class in tqdm(pricer_classes):
    try:
        trade_algo.initialize(pricer_class,ranker_class,classifier_class,start,end,current_date)
        trade_algo.initialize_bench_and_yields()
        trade_algo.initialize_classes()
        trade_algo.load_optimal_parameter()
        simulation = trade_algo.create_current_simulation()
        returns = trade_algo.create_returns(True)
        merged = trade_algo.merge_sim_returns(simulation,returns)
        complete = trade_algo.apply_yields(merged,True)
        trade_algo.initialize_backtester()
        trade_algo.drop_recommendations()
        trades = trade_algo.run_recommendation(complete)
        trade_algo.db.cloud_connect()
        trade_algo.db.store("recs",trades)
        trade_algo.db.disconnect()
    except Exception as e:
        print(trade_algo.name,str(e))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:49<00:00, 49.04s/it]


In [7]:
complete.tail()

,year,quarter,month,week,date,ticker,price_prediction,close,high,low,...,weekly_yield10,monthly_yield10,quarterly_yield10,market_dately_return,projected_dately_return,dately_delta,dately_delta_sign,dately_rrr_tyield1,dately_rrr_tyield2,dately_rrr_tyield10
68249,2023,3.0,7.0,31.0,2023-07-31,YUM,135.943500,137.67,137.94,135.740,...,1.000749,1.00325,1.009781,1.000383,-0.012541,0.012541,-1.0,0.000114,0.000100,0.000073
68250,2023,3.0,7.0,31.0,2023-07-31,ZBH,141.418000,138.15,139.31,137.675,...,1.000749,1.00325,1.009781,1.000383,0.023655,0.023655,1.0,0.000101,0.000086,0.000058
68251,2023,3.0,7.0,31.0,2023-07-31,ZBRA,305.162000,307.96,308.04,299.350,...,1.000749,1.00325,1.009781,1.000383,-0.009086,0.009086,-1.0,0.000105,0.000090,0.000062
68252,2023,3.0,7.0,31.0,2023-07-31,ZION,33.245750,38.25,39.11,37.650,...,1.000749,1.00325,1.009781,1.000383,-0.130830,0.130830,-1.0,-0.000123,-0.000150,-0.000201
68253,2023,3.0,7.0,31.0,2023-07-31,ZTS,176.604218,188.09,190.09,186.440,...,1.000749,1.00325,1.009781,1.000383,-0.061065,0.061065,-1.0,0.000134,0.000121,0.000096
